# Project Goal:

The Heart Attack Analysis & Prediction Dataset by RASHIK RAHMAN on Kaggle provides a vital dataset on medical information for patients with low and high chances of heart attack. By analyzing the learning algorithms, Decision Trees (DT), Logistic Regression (LR), and k-Nearest Neighbors (KNN), we can develop a predictive model that determines the best algorithm for this dataset. This project will ultima
Our dataset comprises crucial medical information about patients, enabling us to assess the likelihood of a heart attack occurrence. By leveraging various machine learning algorithms such as Logistic Regression (LR), Support Vector Machines (SVM), k-Nearest Neighbors (KNN), and Decision Trees (DT), our goal is to develop a robust predictive model. Through this exploration, we aim to determine which algorithm best suits our dataset, ultimately contributing to the ongoing efforts in cardiovascular health prediction.

